<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="45%" align="right" border="4">

# Fourier-based Option Pricing

For several reasons, it is beneficial to have available alternative valuation and pricing approaches to the Monte Carlo simulation approach. One application area is to **benchmark Monte Carlo-based valuation results** against other (potentially more accurate) results. Another area is **model calibration to liquidly traded vanilla instruments** where generally faster numerial methods can be applied.

This part introduces **Fouried-based valuation functions** and benchmarks valuation results from the "standard", simulation-based DX Analytics modeling approach to output of those functions. 

In [1]:
import dx
import datetime as dt

## Risk Factors

The examples and benchmarks to follow rely on four different models:

* geometric Brownian motion (Black-Scholes-Merton 1973)
* jump diffusion (Merton 1976)
* stochastic volatility (Heston 1993)
* stochastic volatility jump diffusion (Bates 1996)

For details on these models and the Fourier-based option pricing approach refer to Hilpisch (2015) (cf. http://eu.wiley.com/WileyCDA/WileyTitle/productCd-1119037999.html).

We first define the single **market and valuation environments**.

In [2]:
# constant short rate
r = dx.constant_short_rate('r', 0.01)

In [3]:
# geometric Brownian motion
me = dx.market_environment('me', dt.datetime(2015, 1, 1))
me.add_constant('initial_value', 100.)
me.add_constant('volatility', 0.2)
me.add_constant('final_date', dt.datetime(2015, 12, 31))
me.add_constant('currency', 'EUR')

In [4]:
# jump component
me.add_constant('lambda', 0.4)
me.add_constant('mu', -0.6)
me.add_constant('delta', 0.2)

In [5]:
# stochastic volatiltiy component
me.add_constant('rho', -.5)
me.add_constant('kappa', 5.0)
me.add_constant('theta', 0.02)
me.add_constant('vol_vol', 0.3)

In [6]:
# valuation environment
val_env = dx.market_environment('val_env', dt.datetime(2015, 1, 1))
val_env.add_constant('paths', 55000)
    # 25,000 paths
val_env.add_constant('frequency', 'D')
    # weekly frequency
val_env.add_curve('discount_curve', r)
val_env.add_constant('starting_date', dt.datetime(2015, 1, 1))
val_env.add_constant('final_date', dt.datetime(2015, 12, 31))

In [7]:
# add valuation environment to market environment
me.add_environment(val_env)

Equipped with the single market environments and the valuation environment, we can instantiate the **simulation model objects**.

In [8]:
gbm = dx.geometric_brownian_motion('gbm', me)

In [9]:
jd = dx.jump_diffusion('jd', me)

In [10]:
sv = dx.stochastic_volatility('sv', me)

In [11]:
svjd = dx.stoch_vol_jump_diffusion('svjd', me)

## Plain Vanilla Put and Call Options

Based on the just defined risk factors, we define 8 diffent options---a **European put and call option per risk factor**, respectively.

In [12]:
# market environment for the options
me_option = dx.market_environment('option', dt.datetime(2015, 1, 1))
me_option.add_constant('maturity', dt.datetime(2015, 12, 31))
me_option.add_constant('strike', 100.)
me_option.add_constant('currency', 'EUR')
me_option.add_environment(me)
me_option.add_environment(val_env)

In [13]:
euro_put_gbm = dx.valuation_mcs_european_single('euro_put', gbm, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_gbm = dx.valuation_mcs_european_single('euro_call', gbm, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [14]:
euro_put_jd = dx.valuation_mcs_european_single('euro_put', jd, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_jd = dx.valuation_mcs_european_single('euro_call', jd, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [15]:
euro_put_sv = dx.valuation_mcs_european_single('euro_put', sv, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_sv = dx.valuation_mcs_european_single('euro_call', sv, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [16]:
euro_put_svjd = dx.valuation_mcs_european_single('euro_put', svjd, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_svjd = dx.valuation_mcs_european_single('euro_call', svjd, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

## Valuation Benchmarking

In this sub-section, we benchmark the **Monte Carlo value estimates** against the **Fourier-based pricing results**.

In [17]:
import numpy as np
import pandas as pd

We first define some parameters used throughout.

In [18]:
freq = '2m'  # used for maturity definitions
periods = 3  # number of intervals for maturity grid
strikes = 5  # number of strikes per maturity
initial_value = 100  # initial value for all risk factors
start = 0.8  # lowest strike in percent of spot
end = 1.2  # highest strike in percent of spot
start_date = '2015/3/1'  # start date for simulation/pricing

### Geometric Brownian Motion

We need to initialize the valuation object first.

In [19]:
euro_put_gbm.present_value()
  # method call needed for initialization

7.3972249999999997

There is a **valuation class for European put and call options in the Black-Scholes-Merton model** available called `BSM_european_option`. It is based on the analytical pricing formula for that model and is instantiated as follows:

In [20]:
bsm_option = dx.BSM_european_option('bsm_opt', me_option)

The following routine benchmarks the Monte Carlo value estimates for the **European put option** against the output from the valuation object based on the analytical pricing formula. The results are quite good since this model is quite easy to discretize exactly and therefore generally shows good convergence of the Monte Carlo estimates.

In [23]:
%%time
# European put
print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
    bsm_option.maturity = maturity
    euro_put_gbm.update(maturity=maturity)
    for strike in np.linspace(start, end, strikes) * initial_value:
        T = (maturity - me_option.pricing_date).days / 365.
        euro_put_gbm.update(strike=strike)
        mcs = euro_put_gbm.present_value()
        bsm_option.strike = strike
        ana = bsm_option.put_value()
        print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.2f '
                % (T, strike, mcs, ana, mcs - ana, (mcs - ana) / ana * 100))

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  0.0342 |  0.0338 |  0.0004 |    1.20 
0.244 |  90.000 |  0.6324 |  0.6524 | -0.0200 |   -3.07 
0.244 | 100.000 |  3.7639 |  3.8130 | -0.0491 |   -1.29 
0.244 | 110.000 | 10.6283 | 10.6957 | -0.0674 |   -0.63 
0.244 | 120.000 | 19.7036 | 19.8537 | -0.1501 |   -0.76 
0.411 |  80.000 |  0.1765 |  0.1748 |  0.0017 |    0.99 
0.411 |  90.000 |  1.2933 |  1.3241 | -0.0308 |   -2.33 
0.411 | 100.000 |  4.8694 |  4.8985 | -0.0291 |   -0.59 
0.411 | 110.000 | 11.3617 | 11.4275 | -0.0659 |   -0.58 
0.411 | 120.000 | 19.9077 | 20.0325 | -0.1248 |   -0.62 
0.578 |  80.000 |  0.3974 |  0.3917 |  0.0057 |    1.44 
0.578 |  90.000 |  1.9205 |  1.9466 | -0.0261 |   -1.34 
0.578 | 100.000 |  5.7326 |  5.7593 | -0.0267 |   -0.46 
0.578 | 110.000 | 12.0342 | 12.0934 | -0.0592 |   -0.49 
0.578 | 120.000 | 20.2201 | 20.3153 | -0.0952 |   -0.47 
CPU times: user 11.9 s, sys: 4.19 s, total: 16.1 s
Wall time: 16.2 s


The same now for the **European call option**.

In [24]:
euro_call_gbm.present_value()
  # method call needed for initialization

8.4449520000000007

In [25]:
%%time
# European calls
print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
    euro_call_gbm.update(maturity=maturity)
    for strike in np.linspace(start, end, strikes) * initial_value:
        T = (maturity - me_option.pricing_date).days / 365.
        euro_call_gbm.update(strike=strike)
        mcs = euro_call_gbm.present_value()
        bsm_option.strike = strike
        bsm_option.maturity = maturity
        ana = bsm_option.call_value()
        print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.2f ' \
                % (T, strike, mcs, ana, mcs - ana, (mcs - ana) / ana * 100))

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 20.0746 | 20.2286 | -0.1540 |   -0.76 
0.244 |  90.000 | 10.8005 | 10.8716 | -0.0712 |   -0.65 
0.244 | 100.000 |  4.0445 |  4.0565 | -0.0120 |   -0.30 
0.244 | 110.000 |  0.9643 |  0.9636 |  0.0008 |    0.08 
0.244 | 120.000 |  0.1444 |  0.1460 | -0.0016 |   -1.10 
0.411 |  80.000 | 20.3867 | 20.5029 | -0.1162 |   -0.57 
0.411 |  90.000 | 11.6013 | 11.6932 | -0.0919 |   -0.79 
0.411 | 100.000 |  5.2563 |  5.3086 | -0.0524 |   -0.99 
0.411 | 110.000 |  1.8634 |  1.8787 | -0.0153 |   -0.81 
0.411 | 120.000 |  0.5086 |  0.5246 | -0.0160 |   -3.06 
0.578 |  80.000 | 20.7657 | 20.8528 | -0.0871 |   -0.42 
0.578 |  90.000 | 12.4304 | 12.4654 | -0.0350 |   -0.28 
0.578 | 100.000 |  6.2735 |  6.3357 | -0.0622 |   -0.98 
0.578 | 110.000 |  2.7331 |  2.7274 |  0.0056 |    0.21 
0.578 | 120.000 |  1.0001 |  1.0070 | -0.0069 |   -0.68 
CPU times: user 11.8 s, sys: 4.15 s, total: 16 s
Wall time: 16 s


### Benchmarking Function

All other valuation benchmarks are generated with **Fourier-based pricing functions** for which the handling is identical. We therefore use the following function for the benchmarks from now on:

In [26]:
def valuation_benchmarking(valuation_object, fourier_function):
    print('%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel'))
    for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
        valuation_object.update(maturity=maturity)
        me_option.add_constant('maturity', maturity)
        for strike in np.linspace(start, end, strikes) * initial_value:
            T = (maturity - me_option.pricing_date).days / 365.
            valuation_object.update(strike=strike)
            mcs = valuation_object.present_value()
            me_option.add_constant('strike', strike)
            fou = fourier_function(me_option)
            print('%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.3f '
                % (T, strike, mcs, fou, mcs - fou, (mcs - fou) / fou * 100))

### Jump Diffusion

The next model is the jump diffusion as proposed by **Merton (1976)**.

In [27]:
euro_put_jd.present_value()
  # method call needed for initialization

14.086788

There is a Fourier-based pricing function available which is called `M76_put_value` and which is used for the benchmarking for the **European put options** that follows.

In [28]:
%time valuation_benchmarking(euro_put_jd, dx.M76_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  2.1509 |  2.1594 | -0.0085 |  -0.394 
0.244 |  90.000 |  3.2076 |  3.2826 | -0.0750 |  -2.285 
0.244 | 100.000 |  5.8238 |  5.8842 | -0.0604 |  -1.027 
0.244 | 110.000 | 11.5824 | 11.6115 | -0.0291 |  -0.251 
0.244 | 120.000 | 19.9121 | 20.0857 | -0.1736 |  -0.864 
0.411 |  80.000 |  3.4752 |  3.4505 |  0.0246 |   0.714 
0.411 |  90.000 |  5.1951 |  5.2162 | -0.0212 |  -0.406 
0.411 | 100.000 |  8.1517 |  8.2266 | -0.0749 |  -0.910 
0.411 | 110.000 | 13.3146 | 13.4430 | -0.1283 |  -0.955 
0.411 | 120.000 | 20.9894 | 20.9238 |  0.0656 |   0.313 
0.578 |  80.000 |  4.5710 |  4.6090 | -0.0379 |  -0.823 
0.578 |  90.000 |  6.8914 |  6.8782 |  0.0132 |   0.192 
0.578 | 100.000 | 10.1158 | 10.2077 | -0.0918 |  -0.900 
0.578 | 110.000 | 15.1332 | 15.2251 | -0.0919 |  -0.603 
0.578 | 120.000 | 21.8842 | 22.0886 | -0.2044 |  -0.925 
CPU times: user 30.9 s, sys: 7.95 s, total: 38.9 s
Wall time: 38.9 s


Accordingly, the benchmarking for the **European call options** based on the Fourier-based `M76_call_value` function.

In [29]:
euro_call_jd.present_value()
  # method call needed for initialization

15.26394

In [30]:
%time valuation_benchmarking(euro_call_jd, dx.M76_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 22.1613 | 22.3543 | -0.1930 |  -0.863 
0.244 |  90.000 | 13.4239 | 13.5018 | -0.0779 |  -0.577 
0.244 | 100.000 |  6.0817 |  6.1277 | -0.0461 |  -0.752 
0.244 | 110.000 |  1.8642 |  1.8794 | -0.0152 |  -0.810 
0.244 | 120.000 |  0.3709 |  0.3780 | -0.0071 |  -1.883 
0.411 |  80.000 | 23.6690 | 23.7786 | -0.1096 |  -0.461 
0.411 |  90.000 | 15.4621 | 15.5853 | -0.1232 |  -0.791 
0.411 | 100.000 |  8.5904 |  8.6367 | -0.0463 |  -0.536 
0.411 | 110.000 |  3.8565 |  3.8941 | -0.0376 |  -0.966 
0.411 | 120.000 |  1.4193 |  1.4160 |  0.0034 |   0.237 
0.578 |  80.000 | 25.0050 | 25.0701 | -0.0651 |  -0.260 
0.578 |  90.000 | 17.3103 | 17.3970 | -0.0866 |  -0.498 
0.578 | 100.000 | 10.7212 | 10.7841 | -0.0629 |  -0.583 
0.578 | 110.000 |  5.8511 |  5.8591 | -0.0080 |  -0.137 
0.578 | 120.000 |  2.8486 |  2.7803 |  0.0683 |   2.456 
CPU times: user 29.5 s, sys: 7.7 s, total: 37.2 s
Wall time: 37.2 s


### Stochastic Volatility

Stochastic volatility models like the one of **Heston (1993)** are popular to reproduce implied volatility smiles observed in markets. First, the benchmarking for the **European put options** using the Fourier-based `H93_put_value` function.

In [31]:
euro_put_sv.present_value()
  # method call needed for initialization

5.295121

In [32]:
%time valuation_benchmarking(euro_put_sv, dx.H93_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  0.0462 |  0.0504 | -0.0043 |  -8.475 
0.244 |  90.000 |  0.5498 |  0.5741 | -0.0243 |  -4.226 
0.244 | 100.000 |  3.2687 |  3.3204 | -0.0517 |  -1.558 
0.244 | 110.000 | 10.1351 | 10.2404 | -0.1053 |  -1.029 
0.244 | 120.000 | 19.5833 | 19.7354 | -0.1520 |  -0.770 
0.411 |  80.000 |  0.1439 |  0.1603 | -0.0164 | -10.248 
0.411 |  90.000 |  0.9533 |  1.0063 | -0.0530 |  -5.262 
0.411 | 100.000 |  3.9243 |  4.0234 | -0.0991 |  -2.462 
0.411 | 110.000 | 10.4343 | 10.5487 | -0.1145 |  -1.085 
0.411 | 120.000 | 19.4981 | 19.6293 | -0.1311 |  -0.668 
0.578 |  80.000 |  0.2591 |  0.2838 | -0.0248 |  -8.731 
0.578 |  90.000 |  1.2537 |  1.3631 | -0.1094 |  -8.025 
0.578 | 100.000 |  4.3855 |  4.5467 | -0.1612 |  -3.545 
0.578 | 110.000 | 10.7099 | 10.8389 | -0.1289 |  -1.190 
0.578 | 120.000 | 19.4509 | 19.5767 | -0.1258 |  -0.643 
CPU times: user 31.9 s, sys: 10.6 s, total: 42.5 s
Wall time: 41.5 s


Second, the benchmarking for the **European call options** based on the Fourier-based `H93_call_value` function.

In [33]:
euro_call_sv.present_value()
  # method call needed for initialization

6.290438

In [34]:
%time valuation_benchmarking(euro_call_sv, dx.H93_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 20.0896 | 20.2453 | -0.1557 |  -0.769 
0.244 |  90.000 | 10.6895 | 10.7933 | -0.1038 |  -0.962 
0.244 | 100.000 |  3.4948 |  3.5639 | -0.0691 |  -1.940 
0.244 | 110.000 |  0.4776 |  0.5083 | -0.0307 |  -6.040 
0.244 | 120.000 |  0.0256 |  0.0276 | -0.0020 |  -7.393 
0.411 |  80.000 | 20.3614 | 20.4884 | -0.1270 |  -0.620 
0.411 |  90.000 | 11.2549 | 11.3754 | -0.1205 |  -1.059 
0.411 | 100.000 |  4.3531 |  4.4335 | -0.0804 |  -1.813 
0.411 | 110.000 |  0.9339 |  0.9998 | -0.0659 |  -6.594 
0.411 | 120.000 |  0.1050 |  0.1214 | -0.0164 | -13.469 
0.578 |  80.000 | 20.6332 | 20.7450 | -0.1118 |  -0.539 
0.578 |  90.000 | 11.7528 | 11.8818 | -0.1291 |  -1.086 
0.578 | 100.000 |  4.9477 |  5.1231 | -0.1754 |  -3.424 
0.578 | 110.000 |  1.3431 |  1.4729 | -0.1298 |  -8.816 
0.578 | 120.000 |  0.2358 |  0.2684 | -0.0326 | -12.144 
CPU times: user 32.2 s, sys: 10.7 s, total: 42.9 s
Wall time: 41.9 s


### Stochastic Volatility Jump-Diffusion

Finally, we consider the combination of the stochastic volatility and jump diffusion models from before as proposed by **Bates (1996)**. The Fourier-based pricing function for **European put options** is called `B96_put_value`.

In [35]:
euro_put_svjd.present_value()
  # method call needed for initialization

13.013221

In [36]:
%time valuation_benchmarking(euro_put_svjd, dx.B96_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  2.1539 |  2.1638 | -0.0099 |  -0.460 
0.244 |  90.000 |  3.2472 |  3.2761 | -0.0289 |  -0.882 
0.244 | 100.000 |  5.4397 |  5.5889 | -0.1493 |  -2.671 
0.244 | 110.000 | 10.9263 | 11.0733 | -0.1470 |  -1.327 
0.244 | 120.000 | 19.6798 | 19.8344 | -0.1546 |  -0.779 
0.411 |  80.000 |  3.2928 |  3.4366 | -0.1438 |  -4.183 
0.411 |  90.000 |  5.0747 |  5.1339 | -0.0592 |  -1.152 
0.411 | 100.000 |  7.7889 |  7.7747 |  0.0143 |   0.183 
0.411 | 110.000 | 12.4027 | 12.5744 | -0.1717 |  -1.365 
0.411 | 120.000 | 19.8989 | 20.1827 | -0.2838 |  -1.406 
0.578 |  80.000 |  4.3416 |  4.5480 | -0.2064 |  -4.537 
0.578 |  90.000 |  6.5114 |  6.7163 | -0.2049 |  -3.051 
0.578 | 100.000 |  9.5515 |  9.6585 | -0.1070 |  -1.108 
0.578 | 110.000 | 13.7357 | 14.1776 | -0.4420 |  -3.117 
0.578 | 120.000 | 20.6447 | 20.9423 | -0.2976 |  -1.421 
CPU times: user 50.3 s, sys: 14.3 s, total: 1min 4s
Wall time: 1min 3s


The Fourier-based counterpart function for **European call options** is called `B96_call_value`.

In [37]:
euro_call_svjd.present_value()
  # method call needed for initialization

13.949764999999999

In [38]:
%time valuation_benchmarking(euro_call_svjd, dx.B96_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 22.1855 | 22.3587 | -0.1732 |  -0.775 
0.244 |  90.000 | 13.3140 | 13.4953 | -0.1813 |  -1.343 
0.244 | 100.000 |  5.7409 |  5.8325 | -0.0916 |  -1.571 
0.244 | 110.000 |  1.2932 |  1.3411 | -0.0480 |  -3.579 
0.244 | 120.000 |  0.1206 |  0.1266 | -0.0060 |  -4.723 
0.411 |  80.000 | 23.5785 | 23.7647 | -0.1862 |  -0.784 
0.411 |  90.000 | 15.2582 | 15.5030 | -0.2448 |  -1.579 
0.411 | 100.000 |  7.9932 |  8.1848 | -0.1916 |  -2.340 
0.411 | 110.000 |  2.9041 |  3.0255 | -0.1214 |  -4.013 
0.411 | 120.000 |  0.6121 |  0.6749 | -0.0627 |  -9.297 
0.578 |  80.000 | 24.7036 | 25.0091 | -0.3055 |  -1.221 
0.578 |  90.000 | 17.0053 | 17.2351 | -0.2298 |  -1.333 
0.578 | 100.000 |  9.9201 | 10.2349 | -0.3147 |  -3.075 
0.578 | 110.000 |  4.5385 |  4.8117 | -0.2732 |  -5.677 
0.578 | 120.000 |  1.4872 |  1.6340 | -0.1468 |  -8.983 
CPU times: user 50.2 s, sys: 14.1 s, total: 1min 4s
Wall time: 1min 3s


## Sources of Errors

Numerical methods like Monte Carlo simulation might suffer from different **sources of errors**, like for example:

* **discretization error**: every **discretization of a continuous time interval**---or a continuous state space to this end---leads to a so-called discretization error
* **approximation errors**: DX Analytics uses in several places approximative, **Euler-based discretization schemes** (e.g. for performance reasons and to allow for consistent correlation modeling) which are known to be biased
* **numerical errors**: the approximation of a continuous probability distribution by a **finite, discrete set of (pseudo-) random numbers** introduces also errors

**Copyright, License & Disclaimer**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

DX Analytics (the "dx library") is licensed under the GNU Affero General Public License
version 3 or later (see [http://www.gnu.org/licenses/](http://www.gnu.org/licenses/)).

DX Analytics comes with no representations
or warranties, to the extent permitted by applicable law.


<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

[http://tpq.io](http://tpq.io) | [team@tpq.io](mailto:team@tpq.io) | [http://twitter.com/dyjh](http://twitter.com/dyjh)

**Quant Platform** |
[http://quant-platform.com](http://quant-platform.com)

**Derivatives Analytics with Python (Wiley Finance)** |
[http://derivatives-analytics-with-python.com](http://derivatives-analytics-with-python.com)

**Python for Finance (O'Reilly)** |
[http://python-for-finance.com](http://python-for-finance.com)